In [1]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load example DEM

In [ ]:
# Read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
z_vals = z_data.values

plt.imshow(z_vals, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
dem_patch = z_vals[13:,:12]
print(dem_patch.shape)
plt.imshow(dem_patch, cmap='viridis')
plt.show()

In [ ]:
# Guess the xy scale
xy = 100 * np.mgrid[-12:13, -12:13]
xvals = xy[0]
yvals = xy[1]

# Plot the data
fig = go.Figure(data=[go.Surface(z=z_vals, x=xvals, y=yvals, showscale=False)])
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.show()

In [5]:
# def height_fn(x, y):
#     return z_vals[12 + int(y)//100, 12 + int(x)//100]

## Select a smaller patch to work with

In [14]:
dem_patch = z_vals[13:,:12]

xy = 100 * np.mgrid[0:12, 0:12]
xvals = xy[0]
yvals = xy[1]

def height_fn(x, y):
    return dem_patch[int(x)//100, int(y)//100]

In [ ]:
# Pick a ray in 3D
ray_origin = np.array([1000.0, 0.0, 500.0])
ray_direction = np.array([-1.0, 1.0, -1.0])
ray_direction /= np.linalg.norm(ray_direction)
ray_length = 1000

# Plot the ray
fig = go.Figure(data=[go.Surface(z=dem_patch, x=xvals, y=yvals, showscale=False)])
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.add_trace(go.Scatter3d(x=[ray_origin[0], ray_origin[0] + ray_direction[0] * ray_length],
                           y=[ray_origin[1], ray_origin[1] + ray_direction[1] * ray_length],
                           z=[ray_origin[2], ray_origin[2] + ray_direction[2] * ray_length],
                           mode='lines', line=dict(color='red', width=5)))
fig.show()

# Slope-based ray marching

## Single iteration

$\theta$ is max slope angle. $\phi$ is downtilt angle of ray.

Law of sines:
$$\frac{h}{\sin(\phi+\theta)}=\frac{\Delta t}{\sin(90-\theta)}$$
$$\Delta t = \frac{h\sin(90-\theta)}{\sin(\phi+\theta)}$$


In [ ]:
# Slope-based ray marching
theta = np.pi/4   # Max slope angle in radians
ray_direction_2d = ray_direction[:2]
phi = np.arctan2(-ray_direction[2], np.linalg.norm(ray_direction_2d))  # Downtilt angle of ray (0 is horizontal, pi/2 is straight down)

ray_point = ray_origin.copy()

xy = ray_point[:2]
h = ray_point[2] - height_fn(*xy)  # height of ray above ground
delta_t = h * np.sin(np.pi/2 - theta) / np.sin(phi + theta)  # delta along ray to next point

next_ray_point = ray_point + delta_t * ray_direction

# Visualize
fig = go.Figure(data=[go.Surface(z=dem_patch, x=xvals, y=yvals, showscale=False)])
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.add_trace(go.Scatter3d(x=[ray_origin[0], ray_origin[0] + ray_direction[0] * ray_length],
                           y=[ray_origin[1], ray_origin[1] + ray_direction[1] * ray_length],
                           z=[ray_origin[2], ray_origin[2] + ray_direction[2] * ray_length],
                           mode='lines', line=dict(color='red', width=5)))
fig.add_trace(go.Scatter3d(x=[ray_point[0], ray_point[0]],
                           y=[ray_point[1], ray_point[1]],
                           z=[ray_point[2], height_fn(*xy)],
                           mode='lines', line=dict(color='red', width=5, dash='dash')))
fig.add_trace(go.Scatter3d(x=[ray_point[0], next_ray_point[0]],
                            y=[ray_point[1], next_ray_point[1]],
                            z=[height_fn(*xy), next_ray_point[2]],
                            mode='lines', line=dict(color='red', width=5, dash='dash')))
fig.show()

## Multiple iterations

In [ ]:
ray_origin = np.array([1000.0, 0.0, 500.0])
ray_direction = np.array([-1.0, 1.0, -0.3])
ray_direction /= np.linalg.norm(ray_direction)
ray_length = 1500

theta = np.pi/4   # Max slope angle in radians
ray_direction_2d = ray_direction[:2]
phi = np.arctan2(-ray_direction[2], np.linalg.norm(ray_direction_2d))  # Downtilt angle of ray (0 is horizontal, pi/2 is straight down)

ray_point = ray_origin.copy()

fig = go.Figure(data=[go.Surface(z=dem_patch, x=xvals, y=yvals, showscale=False)])
fig.add_trace(go.Scatter3d(x=[ray_origin[0], ray_origin[0] + ray_direction[0] * ray_length],
                           y=[ray_origin[1], ray_origin[1] + ray_direction[1] * ray_length],
                           z=[ray_origin[2], ray_origin[2] + ray_direction[2] * ray_length],
                           mode='lines', line=dict(color='red', width=5), showlegend=False))

while(True):
    xy = ray_point[:2]
    h = ray_point[2] - height_fn(*xy)  # height of ray above ground

    if h < 0 or np.linalg.norm(ray_point - ray_origin) > ray_length:
        break

    delta_t = h * np.sin(np.pi/2 - theta) / np.sin(phi + theta)  # delta along ray to next point

    if delta_t < 1.0:
        break

    next_ray_point = ray_point + delta_t * ray_direction

    fig.add_trace(go.Scatter3d(x=[ray_point[0], ray_point[0]],
                           y=[ray_point[1], ray_point[1]],
                           z=[ray_point[2], height_fn(*xy)],
                           mode='lines', line=dict(color='red', width=5, dash='dash'), showlegend=False))
    fig.add_trace(go.Scatter3d(x=[ray_point[0], next_ray_point[0]],
                                y=[ray_point[1], next_ray_point[1]],
                                z=[height_fn(*xy), next_ray_point[2]],
                                mode='lines', line=dict(color='red', width=5, dash='dash'), showlegend=False))
    
    ray_point = next_ray_point

ray_point

fig.add_trace(go.Scatter3d(x=[ray_origin[0]], y=[ray_origin[1]], z=[ray_origin[2]], mode='markers', marker=dict(size=10, color='red')))
fig.add_trace(go.Scatter3d(x=[ray_point[0]], y=[ray_point[1]], z=[ray_point[2]], mode='markers', marker=dict(size=10, color='green')))
fig.update_layout(width=1600, height=900, scene_aspectmode='data')
fig.show()